# Experiment plotting

This notebook contains the code for plotting results for the different experiments.
When run for the first time for a town, condensed summary files are being created which strongly speed up subsequent generations of plots from the same summaries. It is possible to create the plots only from the condensed summaries located in 'summaries/condensed_summaries'.
Note that this works for all experiments but the Rt plots, which still require the full summary file.


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import pickle
import itertools
from lib.measures import *
from lib.experiment import Experiment, Plot, Result, get_properties, load_summary_list, load_summary
from lib.data import collect_data_from_df
from lib.calibrationSettings import calibration_lockdown_dates, calibration_start_dates, calibration_mob_paths
from lib.calibrationFunctions import get_calibrated_params, downsample_cases
import lib.plot as lib_plot
from lib.plot import Plotter
import matplotlib.pyplot as plt
from lib.summary import load_condensed_summary, get_tracing_probability
from matplotlib import colors
import matplotlib
from collections import defaultdict

In [3]:
# commithash = '9ee166f'
commithash = 'a575154'

make_paper_plots = False

# Reduction of different quantities compared to baseline

In [4]:
def plot_relative_reduction(*, country, area, mode, ps_adoption, beacon_proportions, beacon_mode='visit_freq', 
                            manual_tracing=True, show_reduction=True, show_spect_with_beacons=False,
                            isolation_cap=None, ylim=None, commithash=None):
    
    if manual_tracing:
        p_recall = 0.1
        p_manual_reachability = 0.5
    else:
        p_recall = 0.0
        p_manual_reachability = 0.0
        
    if mode == 'r_eff':
        if area == 'TU':
            from lib.settings.town_settings_tubingen import town_population
        else:
            raise NotImplementedError('Specify town population')
    else:
        town_population = None
        
    if isolation_cap is not None:
        is_site_info = '-siteinfo'
        extension = f'-isolation_cap={isolation_cap}'
    else:
        is_site_info = ''
        extension = ''
        

        
    if ylim is None:
        if mode == 'r_eff' and show_reduction:
            ylim = (0.0, 50)
        elif mode == 'r_eff' and not show_reduction:
            ylim= (1.0, 3.5)
            
    
    plot_filename = (f'comparison{is_site_info}-{mode}-{country}-{area}'
                    f'-reduction={show_reduction}'
                    f'-beacon_mode={beacon_mode}'
                    f'-manual_tracing={manual_tracing}'
                    f'-p2p+beacons={show_spect_with_beacons}')
    paths = [
                [f'PanCast/spect-tracing{is_site_info}-{country}-{area}-{commithash}/'
                 f'spect-tracing{is_site_info}-{country}-{area}'
                 f'-p_adoption={p_adoption}'
                 f'-p_recall={p_recall}'
                 f'-p_manual_reachability={p_manual_reachability}'
                 f'-delta_manual_tracing={delta_manual_tracing}'
                 f'{extension}'
                 '.pk' for p_adoption in ps_adoption]
            ]
    titles = ['SPECTs']
    
    for beacon_proportion in beacon_proportions:
        paths.append([f'PanCast/pancast-tracing{is_site_info}-{country}-{area}-{commithash}/'
                      f'pancast-tracing{is_site_info}-{country}-{area}'
                      f'-p_adoption={p_adoption}'
                      f'-beacon_proportion={beacon_proportion}'
                      f'-beacon_mode={beacon_mode}'
                      f'-p2p_beacon={show_spect_with_beacons}'
                      f'-p_recall={p_recall}'
                      f'-p_manual_reachability={p_manual_reachability}'
                      f'-delta_manual_tracing={delta_manual_tracing}'
                      f'{extension}'
                      '.pk' for p_adoption in ps_adoption])
                     
        titles.append(f'PanCast, beacons at {int(beacon_proportion*100)}\% of sites')
    
    
    if show_spect_with_beacons:
        titles = [f'SPECTs + beacons at {int(beacon_proportion*100)}\% of sites' for beacon_proportion in ([0] + beacon_proportions[::-1])]
        new_paths = paths[1:]
        paths = [paths[0]] + new_paths[::-1]
        colors = ['#377eb8', '#fecc5c', '#fd8d3c', '#f03b20', '#bd0026']
    else:
        colors = None


    baseline_path = (f'PanCast/spect-tracing{siteinfo}-{country}-{area}-{commithash}/'
                     f'spect-tracing-{country}-{area}'
                     f'-p_adoption=0.0'
                     f'-p_recall={p_recall}'
                     f'-p_manual_reachability={p_manual_reachability}'
                     f'-delta_manual_tracing={delta_manual_tracing}'
                     f'{extension}'
                     f'.pk')


    plotter = Plotter()
    plotter.compare_peak_reduction(path_list=paths, 
                                   baseline_path=baseline_path, 
                                   ps_adoption=ps_adoption,
                                   area_population=town_population,
                                   titles=titles,
                                   mode=mode,
                                   show_reduction=show_reduction,
                                   log_xscale=True,
                                   ylim=ylim,
                                   colors=colors,
                                   filename=plot_filename,
                                   figsize=lib_plot.FIG_SIZE_FULL_PAGE_DOUBLE_ARXIV_TALL, 
                                   figformat='neurips-double',
                                   legend_is_left=True)


    
if make_paper_plots:
    # Main results section, manual tracing panel
    plot_relative_reduction(
        country='GER', 
        area='TU', 
        mode='cumu_infected',
        ps_adoption=[1.0, 0.5, 0.25, 0.1, 0.05],
        beacon_proportions=[1.0, 0.25, 0.1, 0.05],
        beacon_mode='visit_freq',
        manual_tracing=True,
        show_spect_with_beacons=False,
        commithash=commithash
    )
    
    # Main results section, environmental information
    plot_relative_reduction(
        country='GER', 
        area='TU', 
        mode='cumu_infected',
        ps_adoption=[1.0, 0.5, 0.25, 0.1, 0.05],
        beacon_proportions=[1.0, 0.25, 0.1, 0.05],
        isolation_cap=0.05, # TUNE THIS
        manual_tracing=True, # DISCUSS
        show_spect_with_beacons=False,
        commithash=commithash
    )
    
    # SI results section, manual tracing sensitivity analysis, PanCast, Spects without manual
    plot_relative_reduction(
        country='GER', 
        area='TU', 
        mode='cumu_infected',
        ps_adoption=[1.0, 0.5, 0.25, 0.1, 0.05],
        beacon_proportions=[1.0, 0.25, 0.1, 0.05],
        manual_tracing=False,
        show_spect_with_beacons=False,
        commithash=commithash
    )
    
    # SI results section, interoperability
    plot_relative_reduction(
        country='GER', 
        area='TU', 
        mode='cumu_infected',
        ps_adoption=[1.0, 0.5, 0.25, 0.1, 0.05],
        beacon_proportions=[1.0, 0.25, 0.1, 0.05],
        manual_tracing=True,
        show_spect_with_beacons=True,
        commithash=commithash
    )

else:
    
    plot_relative_reduction(
        country='GER', 
        area='TU', 
        mode='cumu_infected',   # Allowed values ['cumu_infected', 'hosp', 'dead', 'r_eff']
        ps_adoption=[1.0, 0.5, 0.25, 0.1, 0.05],
        beacon_proportions=[1.0, 0.25, 0.1, 0.05],#, 0.02],
        manual_tracing=True,
        show_spect_with_beacons=True,
        show_reduction=True,
        commithash=commithash
    )

NameError: name 'delta_manual_tracing' is not defined

## SI: Environmental information, isolation caps

In [ ]:
def plot_relative_reduction_isolation_cap(*, country, area, mode, ps_adoption, beacon_proportion, beacon_mode='visit_freq', 
                            manual_tracing=True, show_reduction=True, show_spect_with_beacons=False,
                            isolation_caps=None, ylim=None, commithash=None):
    
    if manual_tracing:
        p_recall = 0.1
        p_manual_reachability = 0.5
    else:
        p_recall = 0.0
        p_manual_reachability = 0.0
        
    if mode == 'r_eff':
        if area == 'TU':
            from lib.settings.town_settings_tubingen import town_population
        else:
            raise NotImplementedError('Specify town population')
    else:
        town_population = None
        
        
    if ylim is None:
        if mode == 'r_eff' and show_reduction:
            ylim = (0.0, 50)
        elif mode == 'r_eff' and not show_reduction:
            ylim= (1.0, 3.5)
            
    
    plot_filename = (f'comparison-isolation-caps-{mode}-{country}-{area}'
                    f'-reduction={show_reduction}'
                    f'-beacon_mode={beacon_mode}'
                    f'-manual_tracing={manual_tracing}'
                    f'-p2p+beacons={show_spect_with_beacons}')
    
    
    paths = []
    titles = []
    
    for cap in isolation_caps:
        paths = [
                    [f'PanCast/spect-tracing-siteinfo-{country}-{area}-{commithash}/'
                     f'spect-tracing{is_site_info}-{country}-{area}'
                     f'-p_adoption={p_adoption}'
                     f'-p_recall={p_recall}'
                     f'-p_manual_reachability={p_manual_reachability}'
                     f'-delta_manual_tracing={delta_manual_tracing}'
                     f'-isolation_cap={cap}'
                     '.pk' for p_adoption in ps_adoption]
                ]
        titles.append(f'SPECTS, isolation cap {int(cap*100)}\%')
    
    for cap in isolation_caps:
        paths.append([f'PanCast/pancast-tracing-siteinfo-{country}-{area}-{commithash}/'
                      f'pancast-tracing{is_site_info}-{country}-{area}'
                      f'-p_adoption={p_adoption}'
                      f'-beacon_proportion={beacon_proportion}'
                      f'-beacon_mode={beacon_mode}'
                      f'-p2p_beacon={show_spect_with_beacons}'
                      f'-p_recall={p_recall}'
                      f'-p_manual_reachability={p_manual_reachability}'
                      f'-delta_manual_tracing={delta_manual_tracing}'
                      f'-isolation_cap={cap}'
                      '.pk' for p_adoption in ps_adoption])
                     
        titles.append(f'PanCast, {int(beacon_proportion*100)}\% beacons, isolation cap {int(cap*100)}\%')


    baseline_path = (f'PanCast/spect-tracing-siteinfo-{country}-{area}-{commithash}/'
                     f'spect-tracing-{country}-{area}'
                     f'-p_adoption=0.0'
                     f'-p_recall={p_recall}'
                     f'-p_manual_reachability={p_manual_reachability}'
                     f'-delta_manual_tracing={delta_manual_tracing}'
                     f'-isolation_cap={cap}'
                     f'.pk')


    plotter = Plotter()
    plotter.compare_peak_reduction(path_list=paths, 
                                   baseline_path=baseline_path, 
                                   ps_adoption=ps_adoption,
                                   area_population=town_population,
                                   titles=titles,
                                   mode=mode,
                                   show_reduction=show_reduction,
                                   log_xscale=True,
                                   ylim=ylim,
                                   colors=None,
                                   filename=plot_filename,
                                   figsize=lib_plot.FIG_SIZE_FULL_PAGE_DOUBLE_ARXIV_TALL, 
                                   figformat='neurips-double',
                                   legend_is_left=True)


    
if make_paper_plots:
    # SI results section, environmental information, dependence on isolation caps
    plot_relative_reduction_isolation_cap(
        country='GER', 
        area='TU', 
        mode='cumu_infected',
        ps_adoption=[1.0, 0.5, 0.25, 0.1, 0.05],
        beacon_proportion=0.25,
        isolation_caps=[0.1, 0.05, 0.02, 0.01, 0.005],
        manual_tracing=True,
        commithash=commithash
    )

## SI: Manual tracing delay

In [ ]:
def plot_relative_reduction_manual_tracing_delay(*, country, area, mode, ps_adoption, manual_tracing_delays, 
                                                 beacon_proportion, beacon_mode='visit_freq', 
                                                 show_reduction=True, show_spect_with_beacons=False, 
                                                 ylim=None, commithash=None):
    
    p_recall = 0.1
    p_manual_reachability = 0.5
        
    if mode == 'r_eff':
        if area == 'TU':
            from lib.settings.town_settings_tubingen import town_population
        else:
            raise NotImplementedError('Specify town population')
    else:
        town_population = None
        
        
    if ylim is None:
        if mode == 'r_eff' and show_reduction:
            ylim = (0.0, 50)
        elif mode == 'r_eff' and not show_reduction:
            ylim= (1.0, 3.5)
            
    
    plot_filename = (f'comparison-manual-tracing_delay-{mode}-{country}-{area}'
                    f'-reduction={show_reduction}')
    
    
    paths = []
    titles = []
    
    for delta_manual_tracing in manual_tracing_delays:
        paths = [
                    [f'PanCast/spect-tracing-{country}-{area}-{commithash}/'
                     f'spect-tracing{is_site_info}-{country}-{area}'
                     f'-p_adoption={p_adoption}'
                     f'-p_recall={p_recall}'
                     f'-p_manual_reachability={p_manual_reachability}'
                     f'-delta_manual_tracing={delta_manual_tracing}'
                     '.pk' for p_adoption in ps_adoption]
                ]
        titles.append(f'SPECTS, {int(delta_manual_tracing)} hours delay')
    
    for delta_manual_tracing in manual_tracing_delays:
        paths.append([f'PanCast/pancast-tracing-{country}-{area}-{commithash}/'
                      f'pancast-tracing{is_site_info}-{country}-{area}'
                      f'-p_adoption={p_adoption}'
                      f'-beacon_proportion={beacon_proportion}'
                      f'-beacon_mode={beacon_mode}'
                      f'-p2p_beacon={show_spect_with_beacons}'
                      f'-p_recall={p_recall}'
                      f'-p_manual_reachability={p_manual_reachability}'
                      f'-delta_manual_tracing={delta_manual_tracing}'
                      '.pk' for p_adoption in ps_adoption])
                     
        titles.append(f'PanCast, {int(beacon_proportion*100)}\% beacons, {int(delta_manual_tracing)} hours delay')


    baseline_path = (f'PanCast/spect-tracing-{country}-{area}-{commithash}/'
                     f'spect-tracing-{country}-{area}'
                     f'-p_adoption=0.0'
                     f'-p_recall=0.0'
                     f'-p_manual_reachability=0.0'
                     f'-delta_manual_tracing=0.0'
                     f'.pk')


    plotter = Plotter()
    plotter.compare_peak_reduction(path_list=paths, 
                                   baseline_path=baseline_path, 
                                   ps_adoption=ps_adoption,
                                   area_population=town_population,
                                   titles=titles,
                                   mode=mode,
                                   show_reduction=show_reduction,
                                   log_xscale=True,
                                   ylim=ylim,
                                   colors=None,
                                   filename=plot_filename,
                                   figsize=lib_plot.FIG_SIZE_FULL_PAGE_DOUBLE_ARXIV_TALL, 
                                   figformat='neurips-double',
                                   legend_is_left=True)


    
if make_paper_plots:
    # SI results section, effect of manual tracing delays compared to no manual tracing baseline
    plot_relative_reduction_isolation_cap(
        country='GER', 
        area='TU', 
        mode='cumu_infected',
        ps_adoption=[1.0, 0.5, 0.25, 0.1, 0.05],
        beacon_proportion=0.25,
        manual_tracing_delays=[0.0, 12.0, 24.0],
        commithash=commithash
    )

# Main results: strategic beacon location

In [ ]:
def plot_relative_quantity_heatmap_beacon_mode(*, country, area, mode,
    ps_adoption, beacon_proportions, interpolate, cmap):
    
    filename = (
        f'relative-{mode}-heatmap-pancast'
    )
        
        
    baseline_path = (f'PanCast/spect-tracing-{country}-{area}-{commithash}/'
                 f'spect-tracing-{country}-{area}'
                 f'-p_adoption=0.0'
                 f'-p_recall=0.1'
                 f'-p_manual_reachability=0.5'
                 f'-delta_manual_tracing=0.0'
                 f'.pk')
    
    # pair of list of tuples (x, y, path)
    paths = (
        # spect
        [(beacon_proportion, p_adoption, 
         (f'PanCast/pancast-tracing-{country}-{area}-{commithash}/'
          f'pancast-tracing-{country}-{area}'
          f'-p_adoption={p_adoption}'
          f'-beacon_proportion={beacon_proportion}'
          f'-beacon_mode=random'
          f'-p2p_beacon=False'
          f'-p_recall=0.1'
          f'-p_manual_reachability=0.5'
          f'-delta_manual_tracing=0.0'
          f'.pk'))
         
        for p_adoption in ps_adoption
        for beacon_proportion in beacon_proportions],
        
        # pancast
        [(beacon_proportion, p_adoption, 
         (f'PanCast/pancast-tracing-{country}-{area}-{commithash}/'
          f'pancast-tracing-{country}-{area}'
          f'-p_adoption={p_adoption}'
          f'-beacon_proportion={beacon_proportion}'
          f'-beacon_mode=visit_freq'
          f'-p2p_beacon=False'
          f'-p_recall=0.1'
          f'-p_manual_reachability=0.5'
          f'-delta_manual_tracing=0.0'
          f'.pk'))
         
        for p_adoption in ps_adoption
        for beacon_proportion in beacon_proportions]
    )

    # plots
    plotter = Plotter()
    plotter.relative_quantity_heatmap(
        xlabel=r'\% sites with beacon',
        ylabel=r'\% adoption',
        path_labels=['placed randomly', 'by integr. visit time'],
        paths=paths, 
        mode=mode,
        baseline_path=baseline_path,
        filename=filename, 
#         figsize=lib_plot.FIG_SIZE_FULL_PAGE_DOUBLE_ARXIV_TALL, 
        figsize=(4.0, 2.0), 
        figformat='neurips-double',
        interpolate=interpolate,
        width_ratio=1,
        cmap=cmap,
    )

    
for mode in ["cumu_infected"]:
    plot_relative_quantity_heatmap_beacon_mode(
        mode=mode,
        country='GER', 
        area='TU', 
        ps_adoption=[0.1, 0.25, 0.5, 1.0],
        beacon_proportions=[1.0, 0.25, 0.1, 0.05, 0.02],
        interpolate='linear',
        cmap=plt.cm.RdYlGn,
    )
        

In [ ]:
!bash crop_pdfs.sh plots/relative-*.pdf


# Quantities over time

## Main results, Manual tracing helps with low adoption

In [ ]:
def plot_over_time_section_manual(*, country, area, quantity, mode, beacon_proportions, p_adoption=0.25, ymax, 
                                  rt_plots=True, commithash=None):

    p_recall = 0.1
    p_manual_reachability = 0.5
    delta_manual_tracing = 0.0
    show_spect_with_beacons = False
    beacon_mode = 'visit_freq'

    
    paths = [f'PanCast/spect-tracing-{country}-{area}-{commithash}/'
                     f'spect-tracing-{country}-{area}'
                     f'-p_adoption=0.0'
                     f'-p_recall=0.1'
                     f'-p_manual_reachability=0.5'
                     '.pk',
             
             f'PanCast/spect-tracing-{country}-{area}-{commithash}/'
                 f'spect-tracing-{country}-{area}'
                 f'-p_adoption={p_adoption}'
                 f'-p_recall={p_recall}'
                 f'-p_manual_reachability={p_manual_reachability}'
                 f'-delta_manual_tracing={delta_manual_tracing}'
                 '.pk']
    
    labels = ['Baseline with MT', 'SPECTS']
    
    for beacon_proportion in beacon_proportions:
        paths.append(f'PanCast/pancast-tracing-{country}-{area}'
              f'-p_adoption={p_adoption}'
              f'-beacon_proportion={beacon_proportion}'
              f'-beacon_mode={beacon_mode}'
              f'-p2p_beacon={show_spect_with_beacons}'
              f'-p_recall={p_recall}'
              f'-p_manual_reachability={p_manual_reachability}'
              f'-delta_manual_tracing={delta_manual_tracing}'
              '.pk')
        
        labels.append(f'PanCast, {int(beacon_proportion*100)}\% beacons')
    
    plot_filename = f'section-manual-{mode}-{quantity}_over_time-{country}-{area}'
    plotter = Plotter()

    plotter.compare_quantity(
        paths, 
        titles=labels,
        quantity=quantity,
        mode=mode,
        ymax=ymax,
        filename=plot_filename, 
        figsize=lib_plot.FIG_SIZE_FULL_PAGE_DOUBLE_ARXIV_TALL, 
        figformat='neurips-double',
        legend_is_left=True)
    
    
    if rt_plots and quantity=='infected':
        for i, (path, label) in enumerate(zip(paths, labels)):
            plot_filename = 'rt-main' + label

            # plot
            plotter = Plotter()
            plotter.plot_daily_nbinom_rts(
                path=path, 
                filename=plot_filename,
                cmap_range=(0.5, 1.5),
                figsize=lib_plot.FIG_SIZE_NEURIPS_TRIPLE,
                figformat='neurips-double',
                ymax=3.3,
                label=labels[i],
                #xlim=(0, 185),
                x_axis_dates=False,
                subplots_adjust={'bottom':0.2, 'top': 0.98, 'left': 0.12, 'right': 0.96},
            )
    
    
    
for quantity in ['infected', 'hosp']:
    plot_over_time_section_manual(
        country='GER', 
        area='TU',
        quantity=quantity,    # Allowed values: ['infected', 'hosp', 'dead']
        mode='total',           # Allowed values: ['total', 'daily', 'cumulative', 'weekly incidence']
        ymax=20000, 
        p_adoption=0.25,
        beacon_proportions=[0.5, 0.25, 0.1],
    )

## SI no manual tracing experiment

In [ ]:
def plot_over_time_si_no_manual(*, country, area, quantity, mode, beacon_proportion, p_adoption=0.25, ymax, 
                                  rt_plots=True, commithash=None):

    p_recall = 0.1
    p_manual_reachability = 0.5
    delta_manual_tracing = 0.0
    show_spect_with_beacons = False
    beacon_mode = 'visit_freq'

    
    paths = [f'PanCast/spect-tracing-{country}-{area}-{commithash}/'
                     f'spect-tracing-{country}-{area}'
                     f'-p_adoption=0.0'
                     f'-p_recall=0.0'
                     f'-p_manual_reachability=0.0'
                     f'-delta_manual_tracing=0.0'
                     f'.pk',
             
             f'PanCast/spect-tracing-{country}-{area}-{commithash}/'
                     f'spect-tracing-{country}-{area}'
                     f'-p_adoption=0.0'
                     f'-p_recall={p_recall}'
                     f'-p_manual_reachability={p_manual_reachability}'
                     f'-delta_manual_tracing={delta_manual_tracing}'
                     f'.pk'
        
            f'PanCast/spect-tracing-{country}-{area}-{commithash}/'
                 f'spect-tracing-{country}-{area}'
                 f'-p_adoption={p_adoption}'
                 f'-p_recall=0.0'
                 f'-p_manual_reachability=0.0'
                 f'-delta_manual_tracing=0.0'
                 '.pk',
            
            f'PanCast/spect-tracing-{country}-{area}-{commithash}/'
                 f'spect-tracing-{country}-{area}'
                 f'-p_adoption={p_adoption}'
                 f'-p_recall={p_recall}'
                 f'-p_manual_reachability={p_manual_reachability}'
                 f'-delta_manual_tracing={delta_manual_tracing}'
                 '.pk',
            
            f'PanCast/pancast-tracing-{country}-{area}'
              f'-p_adoption={p_adoption}'
              f'-beacon_proportion={beacon_proportion}'
              f'-beacon_mode={beacon_mode}'
              f'-p2p_beacon={show_spect_with_beacons}'
              f'-p_recall=0.0'
              f'-p_manual_reachability=0.0'
              f'-delta_manual_tracing=0.0'
              '.pk',
            
            f'PanCast/pancast-tracing-{country}-{area}'
              f'-p_adoption={p_adoption}'
              f'-beacon_proportion={beacon_proportion}'
              f'-beacon_mode={beacon_mode}'
              f'-p2p_beacon={show_spect_with_beacons}'
              f'-p_recall={p_recall}'
              f'-p_manual_reachability={p_manual_reachability}'
              f'-delta_manual_tracing={delta_manual_tracing}'
              '.pk']
    
    labels = ['Baseline without MT',
              'Baseline with MT',
              'SPECTS without MT',
              'SPECTS with MT',
              f'PanCast, {int(beacon_proportion*100)}\% beacons, without MT'
              f'PanCast, {int(beacon_proportion*100)}\% beacons, with MT'
             ]
    
    plot_filename = f'si-no-manual-{mode}-{quantity}_over_time-{country}-{area}'
    plotter = Plotter()

    plotter.compare_quantity(
        paths, 
        titles=labels,
        quantity=quantity,
        mode=mode,
        ymax=ymax,
        filename=plot_filename, 
        figsize=lib_plot.FIG_SIZE_FULL_PAGE_DOUBLE_ARXIV_TALL, 
        figformat='neurips-double',
        legend_is_left=True)
    
    if rt_plots and quantity=='infected':
        for i, (path, label) in enumerate(zip(paths, labels)):
            plot_filename = 'rt-no-manual' + label

            # plot
            plotter = Plotter()
            plotter.plot_daily_nbinom_rts(
                path=path, 
                filename=plot_filename,
                cmap_range=(0.5, 1.5),
                figsize=lib_plot.FIG_SIZE_NEURIPS_TRIPLE,
                figformat='neurips-double',
                ymax=3.3,
                label=labels[i],
                #xlim=(0, 185),
                x_axis_dates=False,
                subplots_adjust={'bottom':0.2, 'top': 0.98, 'left': 0.12, 'right': 0.96},
            )
    
    
for quantity in ['infected', 'hosp']:
    plot_over_time_si_no_manual(
        country='GER', 
        area='TU',
        quantity=quantity,    # Allowed values: ['infected', 'hosp', 'dead']
        mode='total',           # Allowed values: ['total', 'daily', 'cumulative', 'weekly incidence']
        ymax=20000, 
        p_adoption=0.25,
        beacon_proportion=0.25,
    )

## SI: Manual tracing delay

In [ ]:
def plot_over_time_si_mt_delay(*, country, area, quantity, mode, manual_tracing_delays,
                               beacon_proportion, p_adoption=0.25, ymax=None, rt_plots=True, commithash=None):

    p_recall = 0.1
    p_manual_reachability = 0.5
    show_spect_with_beacons = False
    beacon_mode = 'visit_freq'

    paths = []
    labels = []
    
    for delta_manual_tracing in manual_tracing_delays:


        paths.append(f'PanCast/spect-tracing-{country}-{area}-{commithash}/'
                     f'spect-tracing-{country}-{area}'
                     f'-p_adoption={p_adoption}'
                     f'-p_recall={p_recall}'
                     f'-p_manual_reachability={p_manual_reachability}'
                     f'-delta_manual_tracing={delta_manual_tracing}'
                     '.pk')
    
        labels = [f'SPECTS, {int(delta_manual_tracing)}h delay']
    
    for delta_manual_tracing in manual_tracing_delays:
        paths.append(f'PanCast/pancast-tracing-{country}-{area}'
              f'-p_adoption={p_adoption}'
              f'-beacon_proportion={beacon_proportion}'
              f'-beacon_mode={beacon_mode}'
              f'-p2p_beacon={show_spect_with_beacons}'
              f'-p_recall={p_recall}'
              f'-p_manual_reachability={p_manual_reachability}'
              f'-delta_manual_tracing={delta_manual_tracing}'
              '.pk')
        
        labels.append(f'PanCast, {int(beacon_proportion*100)}\% beacons, {int(delta_manual_tracing)}h delay')
    
    plot_filename = f'si-manual-delay-{mode}-{quantity}_over_time-{country}-{area}'
    plotter = Plotter()

    plotter.compare_quantity(
        paths, 
        titles=labels,
        quantity=quantity,
        mode=mode,
        ymax=ymax,
        filename=plot_filename, 
        figsize=lib_plot.FIG_SIZE_FULL_PAGE_DOUBLE_ARXIV_TALL, 
        figformat='neurips-double',
        legend_is_left=True)
    
    if rt_plots and quantity=='infected':
        for i, (path, label) in enumerate(zip(paths, labels)):
            plot_filename = 'rt-mt-delay' + label

            # plot
            plotter = Plotter()
            plotter.plot_daily_nbinom_rts(
                path=path, 
                filename=plot_filename,
                cmap_range=(0.5, 1.5),
                figsize=lib_plot.FIG_SIZE_NEURIPS_TRIPLE,
                figformat='neurips-double',
                ymax=3.3,
                label=labels[i],
                #xlim=(0, 185),
                x_axis_dates=False,
                subplots_adjust={'bottom':0.2, 'top': 0.98, 'left': 0.12, 'right': 0.96},
            )
    
    
    
for quantity in ['infected', 'hosp']:
    plot_over_time_si_mt_delay(
        country='GER', 
        area='TU',
        quantity=quantity,    # Allowed values: ['infected', 'hosp', 'dead']
        mode='total',           # Allowed values: ['total', 'daily', 'cumulative', 'weekly incidence']
        ymax=20000, 
        manual_tracing_delays=[0.0, 12.0, 24.0],
        p_adoption=0.25,
        beacon_proportion=0.25,
    )

## SI: Interoperability

In [ ]:
def plot_over_time_si_interoperability(*, country, area, quantity, mode,
                                      beacon_proportion, p_adoption=0.25, ymax=None, rt_plots=True,
                                       commithash=None):


    p_recall = 0.1
    p_manual_reachability = 0.5
    delta_manual_tracing = 0.0
    show_spect_with_beacons = False
    beacon_mode = 'visit_freq'

    paths = [
        # Baseline with MT
        f'PanCast/spect-tracing-{country}-{area}-{commithash}/'
                     f'spect-tracing-{country}-{area}'
                     f'-p_adoption=0.0'
                     f'-p_recall={p_recall}'
                     f'-p_manual_reachability={p_manual_reachability}'
                     f'-delta_manual_tracing={delta_manual_tracing}'
                     f'.pk',
        
        # PanCast
        f'PanCast/pancast-tracing-{country}-{area}'
              f'-p_adoption={p_adoption}'
              f'-beacon_proportion={beacon_proportion}'
              f'-beacon_mode={beacon_mode}'
              f'-p2p_beacon=False'
              f'-p_recall={p_recall}'
              f'-p_manual_reachability={p_manual_reachability}'
              f'-delta_manual_tracing={delta_manual_tracing}'
              '.pk',
        
        # SPECTS
        f'PanCast/spect-tracing-{country}-{area}-{commithash}/'
                     f'spect-tracing-{country}-{area}'
                     f'-p_adoption={p_adoption}'
                     f'-p_recall={p_recall}'
                     f'-p_manual_reachability={p_manual_reachability}'
                     f'-delta_manual_tracing={delta_manual_tracing}'
                     '.pk',
        
        # SPECTS + beacons
        f'PanCast/pancast-tracing-{country}-{area}'
              f'-p_adoption={p_adoption}'
              f'-beacon_proportion={beacon_proportion}'
              f'-beacon_mode={beacon_mode}'
              f'-p2p_beacon=True'
              f'-p_recall={p_recall}'
              f'-p_manual_reachability={p_manual_reachability}'
              f'-delta_manual_tracing={delta_manual_tracing}'
              '.pk',
    ]
    
    labels = ['Baseline with MT',
             'SPECTS',
             f'PanCast, {int(beacon_proportion*100)}\% beacons',
             f'SPECTS + {int(beacon_proportion*100)}\% beacons',
             ]
    

    plot_filename = f'si-interoperability-{mode}-{quantity}_over_time-{country}-{area}'
    plotter = Plotter()

    plotter.compare_quantity(
        paths, 
        titles=labels,
        quantity=quantity,
        mode=mode,
        ymax=ymax,
        filename=plot_filename, 
        figsize=lib_plot.FIG_SIZE_FULL_PAGE_DOUBLE_ARXIV_TALL, 
        figformat='neurips-double',
        legend_is_left=True)
    
    if rt_plots and quantity=='infected':
        for i, (path, label) in enumerate(zip(paths, labels)):
            plot_filename = 'rt-interoperation' + label

            # plot
            plotter = Plotter()
            plotter.plot_daily_nbinom_rts(
                path=path, 
                filename=plot_filename,
                cmap_range=(0.5, 1.5),
                figsize=lib_plot.FIG_SIZE_NEURIPS_TRIPLE,
                figformat='neurips-double',
                ymax=3.3,
                label=labels[i],
                #xlim=(0, 185),
                x_axis_dates=False,
                subplots_adjust={'bottom':0.2, 'top': 0.98, 'left': 0.12, 'right': 0.96},
            )
    
    
    
for quantity in ['infected', 'hosp']:
    plot_over_time_si_interoperability(
        country='GER', 
        area='TU',
        quantity=quantity,    # Allowed values: ['infected', 'hosp', 'dead']
        mode='total',           # Allowed values: ['total', 'daily', 'cumulative', 'weekly incidence']
        ymax=20000, 
        p_adoption=0.25,
        beacon_proportion=0.25,
    )

In [ ]:
# Crop pdf margins
!bash crop_pdfs.sh plots/daily-nbinom-rts-*.pdf
!bash crop_pdfs.sh plots/prob-secondaryCases-*.pdf

## ROC

In [ ]:
def plot_roc(*, country, area, p_adoption, p_beacon, beta_dispersions, manual_tracing, sitetype=None, 
             multiplot=False, commithash=None):
    
    for beta_dispersion in beta_dispersions:
        path = (f'PanCast/beacon-environment-{country}-{area}-{commithash}/'
                          f'beacon-environment-{country}-{area}'
                          f'-beacon=all'
                          f'-p_adoption=1.0'
                          f'-beta_dispersion={beta_dispersion}'
                          '.pk')

        if p_adoption is not None:
            if manual_tracing:
                p_manual_reachability = 0.5
                p_recall = 0.1
            else:
                p_manual_reachability = 0.0
                p_recall = 0.0
        else:
            p_manual_reachability = None
            p_recall = None
            p_beacon = None
            p_adoption = [None]

        if multiplot:
            figsize = (8,3)
            ps_adoption = [p_adoption]
        else:
            figsize = (2.0, 1.8)
            ps_adoption = p_adoption
            
        for p_adoption in ps_adoption:
            print(p_adoption)
            plot_filename = f'ROC-{country}-{area}-p_adoption={p_adoption}-p_beacon={p_beacon}-manual-tracing={manual_tracing}-beta-dispersion={beta_dispersion}'
            plotter = Plotter()

            plotter.plot_roc_curve(
                summaries=None,
                paths=[path],
                filename=plot_filename, 
                p_adoption=p_adoption,
                p_manual_reachability=p_manual_reachability,
                p_recall=p_recall,
                p_beacon=p_beacon,
                sitetype=sitetype,
                action='isolate',
                figsize=figsize, 
    #             figsize=(1.8, 1.6), 
                figformat='neurips-double',
                verbose=False,
                use_medical_labels=True)

    
plot_roc(
    country='GER', 
    area='TU', 
    p_adoption=None,
    p_beacon=None,
    sitetype=None,
    beta_dispersions=[10.0],
    manual_tracing=True,
    multiplot=False,
    commithash='43d4464'
)

plot_roc(
    country='GER', 
    area='TU', 
    p_adoption=[0.5, 0.25, 0.1, 0.05],
    p_beacon=[0.5, 0.25, 0.1, 0.05],
    beta_dispersions=[10.0],
    manual_tracing=True,
    multiplot=True,
    commithash='43d4464'
)
    
# Crop pdf margins
!bash crop_pdfs.sh plots/ROC-*.pdf

In [ ]:
def plot_sens_spec_per_sitetype(beta_dispersion = 10.0, commithash='43d4464'):
    summary = load_condensed_summary(f'PanCast/beacon-environment-GER-TU-{commithash}/'
                              f'beacon-environment-GER-TU'
                              f'-beacon=all'
                              f'-p_adoption=1.0'
                              f'-beta_dispersion={beta_dispersion}'
                              '.pk')
    print()

    tracing_stats = summary['tracing_stats']
    thresholds = list(tracing_stats.keys())
    thres = thresholds[0]

    results = {'sites': defaultdict(lambda: {'fpr': [], 'tpr': []}),
              'no_sites': defaultdict(lambda: {'fpr': [], 'tpr': []})}

    # Compute FPR, TPR
    for sites in ['no_sites', 'sites']:
        for sitetype in tracing_stats[thres].keys():
            for thres in thresholds:
                infected = tracing_stats[thres][sitetype][sites]['isolate']['tp'].mean() + tracing_stats[thres][sitetype][sites]['isolate']['fn'].mean()
                not_infected = tracing_stats[thres][sitetype][sites]['isolate']['tn'].mean() + tracing_stats[thres][sitetype][sites]['isolate']['fp'].mean()
                isolated = tracing_stats[thres][sitetype][sites]['isolate']['tp'].mean() + tracing_stats[thres][sitetype][sites]['isolate']['fp'].mean()
                not_isolated = tracing_stats[thres][sitetype][sites]['isolate']['fn'].mean() + tracing_stats[thres][sitetype][sites]['isolate']['tn'].mean()
                tpr = tracing_stats[thres][sitetype][sites]['isolate']['tp'].mean() / infected
                fpr = tracing_stats[thres][sitetype][sites]['isolate']['fp'].mean() / not_infected
                results[sites][sitetype]['fpr'].append(fpr)
                results[sites][sitetype]['tpr'].append(tpr)

    # Plot per site type
    fig, ax = plt.subplots(1, len(tracing_stats[thres].keys()), figsize=(16,1.5))
    keys = list(tracing_stats[thres].keys())
    keys = np.roll(keys, 1)
    for i, sitetype in enumerate(keys):
        for k, sites in enumerate(['no_sites', 'sites']):
            if sites == 'sites':
                label = 'PanCast'
                color = '#e41a1c'
            else:
                label = 'SPECTS'
                color = '#377eb8'
                
            ax[i].plot(thresholds, 1- np.asarray(results[sites][sitetype]['fpr']), label=label+' specificity', color=color)
            ax[i].plot(thresholds, results[sites][sitetype]['tpr'], label=label+', sensitivity', color=color, linestyle='dashed')
            ax[i].set_xlim([0,0.3])
            title = 'bus stop' if sitetype == 'bus_stop' else sitetype
            title = 'total' if sitetype == 'stats' else title
            ax[i].set_title(title, fontsize=14)
            ax[i].set_xlabel('Isolation threshold', fontsize=11)
            if i== 0:
                ax[i].legend()
    filename = f'site-dependent-characteristics-beta_dispersion={beta_dispersion}'
    plt.savefig('plots/' + filename + '.pdf', format='pdf', facecolor=None,
                        dpi=200, bbox_inches='tight')

    

plot_sens_spec_per_sitetype()

# SI: Manual tracing senitivity heatmaps

In [ ]:
def plot_tracing_probability(mode, p_adoption, ps_beacon):
    fig, axs = plt.subplots(1, 1+len(ps_beacon), figsize=(3 * (1 + len(ps_beacon)), 2.5))

    p_recall, p_manually_reachable = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
    
    # SPECTS
    p_tracable = get_tracing_probability('SPECTS', p_adoption=p_adoption, p_manual_reachability=p_manually_reachable, p_recall=p_recall)
    c = axs[0].pcolormesh(p_manually_reachable, p_recall, p_tracable, cmap='RdYlGn', vmin=0, vmax=1.0)
    axs[0].set_title('SPECTS')
    axs[0].set_ylabel(r'$p_{\rm recall}$', fontsize=14)
    axs[0].set_xlabel(r'$p_{\rm reachablity}$', fontsize=14)
    
    # PanCast
    for k, p_beacon in enumerate(ps_beacon):
        p_tracable = get_tracing_probability('PanCast', p_adoption=p_adoption, p_manual_reachability=p_manually_reachable, p_recall=p_recall, p_beacon=p_beacon)
        c = axs[k+1].pcolormesh(p_manually_reachable, p_recall, p_tracable, cmap='RdYlGn', vmin=0, vmax=1.0)
        axs[k+1].set_title(f'PanCast, {int(p_beacon*100)}% beacons')
        axs[k+1].set_ylabel(r'$p_{\rm recall}$', fontsize=14)
        axs[k+1].set_xlabel(r'$p_{\rm reachablity}$', fontsize=14)

    fig.tight_layout()
    fig.subplots_adjust(right=0.82)

    #cbar_ax = fig.add_axes([0.84, 0.215, 0.03, 0.66])
    cbar_ax = fig.add_axes([0.84, 0.255, 0.025, 0.6])
    cbar = matplotlib.colorbar.ColorbarBase(
        cbar_ax, cmap=plt.cm.RdYlGn,
        norm=colors.Normalize(vmin=0, vmax=1),
        orientation='vertical')
    cbar.set_label('Tracing probablity', labelpad=5.0, fontsize=12)
    
    filename = f'tracing-prob-recall-vs-reachable-p_adoption={p_adoption}'
    plt.savefig('plots/' + filename + '.pdf', format='pdf', facecolor=None, dpi=200)

    plt.show()

for p_adoption in [0.1, 0.25, 0.5]:
    plot_tracing_probability(mode=None, p_adoption=p_adoption, ps_beacon=[1.0, 0.25, 0.1])

In [ ]:
def plot_tracing_probability2(mode, ps_beacon):
    fig, axs = plt.subplots(1, 1+len(ps_beacon), figsize=(3 * (1 + len(ps_beacon)), 2.5))

    p_recall_reachability, p_adoption = np.meshgrid(np.linspace(0, 1, 100), np.linspace(0, 1, 100))
    p_recall = np.sqrt(p_recall_reachability)
    p_reachable = np.sqrt(p_recall_reachability)
    
    # SPECTS
    p_tracable = get_tracing_probability('SPECTS', p_adoption=p_adoption, p_manual_reachability=p_reachable, p_recall=p_recall)
    c = axs[0].pcolormesh(p_adoption, p_recall_reachability, p_tracable, cmap='RdBu', vmin=0.0, vmax=1.0)
    axs[0].set_title('SPECTS')
    axs[0].set_ylabel(r'$p_{\rm recall} \cdot p_{\rm reachablity}$', fontsize=14)
    axs[0].set_xlabel(r'$p_{\rm adoption}$', fontsize=14)
    
    # PanCast
    for k, p_beacon in enumerate(ps_beacon):
        p_tracable = get_tracing_probability('PanCast', p_adoption=p_adoption, p_manual_reachability=p_reachable, p_recall=p_recall, p_beacon=p_beacon)
        c = axs[k+1].pcolormesh(p_adoption, p_recall_reachability, p_tracable, cmap='RdBu', vmin=0.0, vmax=1.0)
        axs[k+1].set_title(f'PanCast, {int(p_beacon*100)}% beacons')
        axs[k+1].set_ylabel(r'$p_{\rm recall} \cdot p_{\rm reachablity}$', fontsize=14)
        axs[k+1].set_xlabel(r'$p_{\rm adoption}$', fontsize=14)

    fig.tight_layout()
    fig.subplots_adjust(right=0.82)

    cbar_ax = fig.add_axes([0.84, 0.255, 0.025, 0.6])
    cbar = matplotlib.colorbar.ColorbarBase(
        cbar_ax, cmap=plt.cm.RdBu,
        norm=colors.Normalize(vmin=0, vmax=1),
        orientation='vertical')
    cbar.set_label('Tracing probablity', labelpad=5.0, fontsize=12)
    
    filename = 'tracing-prob-adoption-vs-manual'
    plt.savefig('plots/' + filename + '.pdf', format='pdf', facecolor=None, dpi=200)

    plt.show()
    
plot_tracing_probability2(mode=None, ps_beacon=[1.0, 0.25, 0.1])

In [ ]:
!bash crop_pdfs.sh plots/tracing-prob-*.pdf

# Beacon locations maps

In [49]:
from lib.mobilitysim import MobilitySimulator
from lib.town_maps import MapIllustrator
import importlib
from folium.folium import Map
from folium import CircleMarker

def show_beacon_distribution(beacon_mode, beacon_proportion):

    beacon_config = dict(mode=beacon_mode, proportion_with_beacon=beacon_proportion)
    mob_settings_file = 'lib/mobility/Tubingen_settings_1.pk'

    with open(mob_settings_file, 'rb') as fp:
        mob_settings = pickle.load(fp)

    mob_settings['beacon_config'] = beacon_config
    mob = MobilitySimulator(**mob_settings)

    ill = MapIllustrator()
    ill.marker_radius = 3.0
    bbox = importlib.import_module(f'lib.settings.town_settings_tubingen').bbox

    # center map around the given bounding box
    center = ((bbox[0]+bbox[1])/2,(bbox[2]+bbox[3])/2)
    m = Map(location=center,tiles=ill.tile)
    m.fit_bounds([(bbox[0],bbox[2]),(bbox[1],bbox[3])])

    # set marker labels as site types
    labels = [mob.site_dict[site] for site in mob.site_type]
    color_map = matplotlib.cm.get_cmap('tab10')

    for i_marker, marker in enumerate(mob.site_loc):

        if not mob.site_has_beacon[i_marker]:
            color = 'darkgrey'
            opacity = 0.5
            fill_opacity = 1.0

            CircleMarker(
                location = marker, 
                radius = ill.marker_radius,
                color=color,
                fill_color=color,
                opacity=opacity,
                fill_opacity=fill_opacity,
                popup=labels[i_marker],
            ).add_to(m)
            
    for i_marker, marker in enumerate(mob.site_loc):

        if mob.site_has_beacon[i_marker]:
            color = matplotlib.colors.rgb2hex(color_map(mob.site_type[i_marker]))
            opacity = 0.7
            fill_opacity = 1.0
            
            CircleMarker(
                location = marker, 
                radius = ill.marker_radius,
                color=color,
                fill_color=color,
                opacity=opacity,
                fill_opacity=fill_opacity,
                popup=labels[i_marker],
            ).add_to(m)
        

    map_name = f'map-{beacon_mode}-beacons={beacon_proportion}'
    if not os.path.exists('maps'):
        os.mkdir('maps')
    m.save('maps/'+map_name+'.html')
    
for beacon_mode in ['visit_freq', 'random']:
    show_beacon_distribution(beacon_mode=beacon_mode, beacon_proportion=0.25)

# Statistics and playground

In [ ]:
def get_tracing_probability2(mode, p_adoption, p_manual_reachability, p_recall, p_beacon=0.0):
    if mode == 'SPECTs':
        p_digital = p_adoption ** 2
        p_manual = p_recall * p_manual_reachability * (1 - p_digital)
        p_tracing = p_digital + p_manual
    elif mode == 'PanCast':
        p_digital = (p_adoption ** 2) * p_beacon
        p_manual = p_recall * p_manual_reachability * (1 - p_digital)
        p_digital_manual = p_beacon * p_adoption * p_manual_reachability * (1 - p_digital - p_manual)
        p_manual_digital = p_beacon * p_adoption * p_recall  * (1 - p_digital - p_manual - p_digital_manual)
        p_tracing = p_digital + p_manual + p_digital_manual + p_manual_digital
        #print(p_digital, p_digital_manual, p_manual_digital, p_manual)
    else:
        NotImplementedError('`mode` can only be in ["SPECTs", "PanCast"]')
    return p_tracing


p_tracable = get_tracing_probability2('PanCast', 
                                     p_adoption=0.9, 
                                     p_manual_reachability=0.9999,
                                     p_recall=0.9999, 
                                     p_beacon=0.9)
print(p_tracable)

In [ ]:
from tabulate import tabulate

def show_roc_statistics(threshold_indices, beta_dispersion=10.0, commithash='baf7764'):

    assert beta_dispersion in [10.0, 'custom']

    summary = load_condensed_summary(f'PanCast/beacon-environment-GER-TU-{commithash}/'
                              f'beacon-environment-GER-TU'
                              f'-beacon=all'
                              f'-p_adoption=1.0'
                              f'-beta_dispersion={beta_dispersion}'
                              '.pk')

    tracing_stats = summary['tracing_stats']
    thresholds = list(tracing_stats.keys())

    betas = {'custom': {'sites': {'total': 0, 'education': 0.02731594725044837, 'social': 0.05463189450089674, 'bus_stop': 0.0018210631500298917, 'office': 0.03642126300059783, 'supermarket': 0.018210631500298915},
                        'no_sites': {'total': 0, 'education': 0.04027511551976204, 'social': 0.04027511551976204, 'bus_stop': 0.04027511551976204, 'office': 0.04027511551976204, 'supermarket': 0.04027511551976204}
                        },
             10.0: {'no_sites': {'total': 0, 'education': 0.04027511551976204, 'social': 0.04027511551976204, 'bus_stop': 0.04027511551976204, 'office': 0.04027511551976204, 'supermarket': 0.04027511551976204},
                    'sites': {'total': 0, 'education': 0.008195195098129743, 'social': 0.08195195098129741, 'bus_stop': 0.0008195195098129743, 'office': 0.008195195098129743, 'supermarket': 0.008195195098129743}
                    }
            }

    betas = betas[beta_dispersion]

    for thres in threshold_indices:
        thres = thresholds[thres]
        print()
        print()
        print('Threshold: ', thres)
        for sites in ['no_sites', 'sites']:
            values = []
            print()
            if sites == 'no_sites':
                print('SPECTS')
            else:
                print('PanCast')
            print()
            for sitetype in tracing_stats[thres].keys():
                infected = tracing_stats[thres][sitetype][sites]['isolate']['tp'].mean() + tracing_stats[thres][sitetype][sites]['isolate']['fn'].mean()
                not_infected = tracing_stats[thres][sitetype][sites]['isolate']['tn'].mean() + tracing_stats[thres][sitetype][sites]['isolate']['fp'].mean()
                isolated = tracing_stats[thres][sitetype][sites]['isolate']['tp'].mean() + tracing_stats[thres][sitetype][sites]['isolate']['fp'].mean()
                not_isolated = tracing_stats[thres][sitetype][sites]['isolate']['fn'].mean() + tracing_stats[thres][sitetype][sites]['isolate']['tn'].mean()
                tpr = tracing_stats[thres][sitetype][sites]['isolate']['tp'].mean() / infected
                fpr = tracing_stats[thres][sitetype][sites]['isolate']['fp'].mean() / not_infected

                if sitetype == 'stats':
                    sitetype = 'total'
                values.append([sitetype, "{:.4f}".format(betas[sites][sitetype]), int(infected), int(not_infected), int(isolated), tpr, fpr])
            print(tabulate(values, headers=['Sitetype', 'beta', 'infected', 'not infected', 'isolated', 'TPR', 'FPR']))
            
show_roc_statistics([10], commithash='43d4464')